In [2]:
from torchvision.datasets import CIFAR10
import numpy as np 
import torchvision.transforms as transforms
import torch 
from torch.utils.data.dataloader import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

In [3]:
## Normalization adapted for CIFAR10
normalize_scratch = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

# Transforms is a list of transformations applied on the 'raw' dataset before the data is fed to the network. 
# Here, Data augmentation (RandomCrop and Horizontal Flip) are applied to each batch, differently at each epoch, on the training set data only
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize_scratch,
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize_scratch,
])

### The data from CIFAR10 will be downloaded in the following folder
rootdir = './data/cifar10'

c10train = CIFAR10(rootdir,train=True,download=True,transform=transform_train)
c10test = CIFAR10(rootdir,train=False,download=True,transform=transform_test)



Files already downloaded and verified
Files already downloaded and verified


In [4]:
## number of target samples for the final dataset
num_train_examples = len(c10train)
num_samples_subset = 15000

## We set a seed manually so as to reproduce the results easily
seed  = 2147483647

batch_size = 256

## Generate a list of shuffled indices ; with the fixed seed, the permutation will always be the same, for reproducibility
indices = list(range(num_train_examples))
np.random.RandomState(seed=seed).shuffle(indices)## modifies the list in place

## We define the Subset using the generated indices 
c10train_subset = torch.utils.data.Subset(c10train,indices[:num_samples_subset])
print(f"Initial CIFAR10 dataset has {len(c10train)} samples")
print(f"Subset of CIFAR10 dataset has {len(c10train_subset)} samples")

# Finally we can define anoter dataloader for the training data
trainloader = DataLoader(c10train,batch_size=batch_size,shuffle=True)
testloader = DataLoader(c10test,batch_size=batch_size) 
trainloader_subset = DataLoader(c10train_subset,batch_size=batch_size,shuffle=True)
### You can now use either trainloader (full CIFAR10) or trainloader_subset (subset of CIFAR10) to train your networks.

Initial CIFAR10 dataset has 50000 samples
Subset of CIFAR10 dataset has 15000 samples


In [5]:
from preactresnet import PreActResNet18
model = PreActResNet18()
model = model.to("cuda:0")
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = CrossEntropyLoss()

x = torch.randn(batch_size,3,32,32, device="cuda:0")
y = model((x))
# print(y.size(), x.size())
loss = loss_fn(y, torch.rand_like(y))
# print(loss.detach().sum())
# print(y.detach().sum())

torch.Size([512, 100])


In [6]:
def train(model, loader, optimizer, loss_fn, n_epochs=1, test):
    for e in range(n_epochs):
        print("epoch", e)
        for i_batch, batch in enumerate(loader):
            optimizer.zero_grad()
            images, labels = batch
            images = images.to("cuda:0")
            labels = labels.to("cuda:0")
            outputs = model.forward(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            if i_batch%10==0: print("batch", i_batch, "training loss" loss.item())

In [8]:
from preactresnet import PreActResNet18
model = PreActResNet18()
model = model.to("cuda:0")
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = CrossEntropyLoss()
train(model, trainloader_subset, optimizer, loss_fn ,n_epochs=3)

epoch 0
batch 0 4.644861698150635
batch 10 2.7299909591674805
batch 20 2.159644842147827
batch 30 1.9690576791763306
batch 40 2.004293441772461
batch 50 1.9540870189666748
epoch 1
batch 0 1.9135969877243042
batch 10 1.8364839553833008
batch 20 1.7753396034240723
batch 30 1.7579840421676636
batch 40 1.761443018913269
batch 50 1.7549067735671997
